In [1]:
import re, cv2
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop

In [2]:
total_sample_size = 560


def get_data(total_sample_size):
    #read the image
    image = cv2.imread('dataset/' + str(1) + '/' + "TD_RGB_E_" + str(1) + '.jpg')
    #reduce the size
    image = cv2.resize(image,(192,252))
    #get the new size
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0
    
    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, dim1, dim2, 3]) # 2 is for pairs
    y_genuine = np.zeros([total_sample_size, 1])
    
    for i in range(112):
        for j in range(int(total_sample_size/112)):
            ind1 = 0
            ind2 = 0
            
            #read images from same directory (genuine pair)
            while ind1 == ind2:
                ind1 = np.random.randint(5)
                ind2 = np.random.randint(5)
            
            # read the two images
            img1 = cv2.imread('dataset/' + str(i+1) + '/' + "TD_RGB_E_" + str(ind1 + 1) + '.jpg')
            img2 = cv2.imread('dataset/' + str(i+1) + '/' + "TD_RGB_E_" + str(ind2 + 1) + '.jpg')
            
            #reduce the size
            img1 = cv2.resize(img1,(192,252))
            img2 = cv2.resize(img2,(192,252))
            #store the images to the initialized numpy array
            x_geuine_pair[count, 0, :, :, :] = img1
            x_geuine_pair[count, 1, :, :, :] = img2
            
            #as we are drawing images from the same directory we assign label as 1. (genuine pair)
            y_genuine[count] = 1
            count += 1

    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, dim1, dim2, 3])
    y_imposite = np.zeros([total_sample_size, 1])
    
    for i in range(int(total_sample_size/5)):
        for j in range(5):
            
            #read images from different directory (imposite pair)
            while True:
                ind1 = np.random.randint(112)
                ind2 = np.random.randint(112)
                if ind1 != ind2:
                    break
                    
            img1 = cv2.imread('dataset/' + str(ind1+1) + '/' + "TD_RGB_E_" + str(j + 1) + '.jpg')
            img2 = cv2.imread('dataset/' + str(ind2+1) + '/' + "TD_RGB_E_" + str(j + 1) + '.jpg')

            img1 = cv2.resize(img1,(192,252))
            img2 = cv2.resize(img2,(192,252))

            x_imposite_pair[count, 0, :, :, :] = img1
            x_imposite_pair[count, 1, :, :, :] = img2
            #as we are drawing images from the different directory we assign label as 0. (imposite pair)
            y_imposite[count] = 0
            count += 1
            
    #now, concatenate, genuine pairs and imposite pair to get the whole data
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y

In [3]:
X, Y = get_data(total_sample_size)
print(X.shape, Y.shape)

(1120, 2, 252, 192, 3) (1120, 1)


In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)

In [5]:
def build_base_network(input_shape):
    
    seq = Sequential()
    
    nb_filter = [6, 12]
    kernel_size = 3
    
    
    #convolutional layer 1
    seq.add(Convolution2D(nb_filter[0], kernel_size, kernel_size, input_shape=input_shape))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))  
    seq.add(Dropout(.25))
    
    #convolutional layer 2
    seq.add(Convolution2D(nb_filter[1], kernel_size, kernel_size))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2))) 
    seq.add(Dropout(.25))

    #flatten 
    seq.add(Flatten())
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(64, activation='relu'))
    return seq

In [6]:
input_dim = x_train.shape[2:]
print(input_dim)

(252, 192, 3)


In [116]:
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

In [117]:
base_network = build_base_network(input_dim)
feat_vecs_a = base_network(img_a)
feat_vecs_b = base_network(img_b)

In [118]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [119]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [120]:
epochs = 30
rms = RMSprop()

In [121]:
model = Model(inputs=[img_a, img_b], outputs=distance)

In [122]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [123]:
#model.compile(loss=contrastive_loss, optimizer=rms)
model.compile(loss="binary_crossentropy", optimizer=rms, metrics=['accuracy'])

In [124]:
img_1 = x_train[:, 0]
img2 = x_train[:, 1]

In [125]:
img_3 = x_test[:, 0]
img4 = x_test[:, 1]

In [126]:
model.fit([img_1, img2], y_train, validation_data=([img_3, img4], y_test),batch_size=512, verbose=1, epochs=2000)

Epoch 1/2000
2/2 [==============================] - 6s 848ms/step - loss: 2.6302 - accuracy: 0.4929 - val_loss: 1.3565 - val_accuracy: 0.5036
Epoch 2/2000
2/2 [==============================] - 1s 364ms/step - loss: 0.7249 - accuracy: 0.4905 - val_loss: 1.3591 - val_accuracy: 0.5036
Epoch 3/2000
2/2 [==============================] - 1s 307ms/step - loss: 0.7254 - accuracy: 0.4702 - val_loss: 1.3595 - val_accuracy: 0.5036
Epoch 4/2000
2/2 [==============================] - 0s 286ms/step - loss: 0.7192 - accuracy: 0.5012 - val_loss: 1.3560 - val_accuracy: 0.5036
Epoch 5/2000
2/2 [==============================] - 1s 455ms/step - loss: 0.7346 - accuracy: 0.4560 - val_loss: 1.3592 - val_accuracy: 0.5036
Epoch 6/2000
2/2 [==============================] - 1s 345ms/step - loss: 0.7180 - accuracy: 0.4714 - val_loss: 1.3483 - val_accuracy: 0.5036
Epoch 7/2000
2/2 [==============================] - 1s 401ms/step - loss: 0.7249 - accuracy: 0.4512 - val_loss: 1.3454 - val_accuracy: 0.5036
Epoch 

2/2 [==============================] - 1s 379ms/step - loss: 0.7088 - accuracy: 0.5036 - val_loss: 1.2434 - val_accuracy: 0.5036
Epoch 59/2000
2/2 [==============================] - 1s 389ms/step - loss: 0.7059 - accuracy: 0.5167 - val_loss: 1.2653 - val_accuracy: 0.5036
Epoch 60/2000
2/2 [==============================] - 1s 350ms/step - loss: 0.7123 - accuracy: 0.4857 - val_loss: 1.2715 - val_accuracy: 0.5036
Epoch 61/2000
2/2 [==============================] - 1s 396ms/step - loss: 0.7073 - accuracy: 0.5131 - val_loss: 1.2141 - val_accuracy: 0.5036
Epoch 62/2000
2/2 [==============================] - 1s 396ms/step - loss: 0.7224 - accuracy: 0.4964 - val_loss: 1.2529 - val_accuracy: 0.5036
Epoch 63/2000
2/2 [==============================] - 1s 380ms/step - loss: 0.7096 - accuracy: 0.4917 - val_loss: 1.2398 - val_accuracy: 0.5036
Epoch 64/2000
2/2 [==============================] - 1s 354ms/step - loss: 0.7184 - accuracy: 0.4702 - val_loss: 1.2611 - val_accuracy: 0.5036
Epoch 65/2000

2/2 [==============================] - 1s 384ms/step - loss: 0.7114 - accuracy: 0.4857 - val_loss: 1.1627 - val_accuracy: 0.5036
Epoch 116/2000
2/2 [==============================] - 1s 379ms/step - loss: 0.7169 - accuracy: 0.5000 - val_loss: 1.1817 - val_accuracy: 0.5036
Epoch 117/2000
2/2 [==============================] - 1s 373ms/step - loss: 0.7092 - accuracy: 0.4893 - val_loss: 1.1821 - val_accuracy: 0.5036
Epoch 118/2000
2/2 [==============================] - 1s 374ms/step - loss: 0.7068 - accuracy: 0.4976 - val_loss: 1.1882 - val_accuracy: 0.5036
Epoch 119/2000
2/2 [==============================] - 1s 343ms/step - loss: 0.7084 - accuracy: 0.5071 - val_loss: 1.1733 - val_accuracy: 0.5036
Epoch 120/2000
2/2 [==============================] - 1s 405ms/step - loss: 0.7055 - accuracy: 0.4917 - val_loss: 1.1690 - val_accuracy: 0.5036
Epoch 121/2000
2/2 [==============================] - 1s 387ms/step - loss: 0.6974 - accuracy: 0.5190 - val_loss: 1.1978 - val_accuracy: 0.5036
Epoch 1

Epoch 172/2000
2/2 [==============================] - 1s 398ms/step - loss: 0.7066 - accuracy: 0.5095 - val_loss: 1.1012 - val_accuracy: 0.5036
Epoch 173/2000
2/2 [==============================] - 1s 375ms/step - loss: 0.7051 - accuracy: 0.4964 - val_loss: 1.1065 - val_accuracy: 0.5036
Epoch 174/2000
2/2 [==============================] - 1s 377ms/step - loss: 0.6973 - accuracy: 0.5167 - val_loss: 1.1245 - val_accuracy: 0.5036
Epoch 175/2000
2/2 [==============================] - 1s 396ms/step - loss: 0.6917 - accuracy: 0.5310 - val_loss: 1.0843 - val_accuracy: 0.5036
Epoch 176/2000
2/2 [==============================] - 1s 379ms/step - loss: 0.7072 - accuracy: 0.4952 - val_loss: 1.1082 - val_accuracy: 0.5036
Epoch 177/2000
2/2 [==============================] - 1s 345ms/step - loss: 0.7084 - accuracy: 0.4917 - val_loss: 1.1057 - val_accuracy: 0.5036
Epoch 178/2000
2/2 [==============================] - 1s 391ms/step - loss: 0.6879 - accuracy: 0.5310 - val_loss: 1.0873 - val_accuracy:

Epoch 229/2000
2/2 [==============================] - 1s 400ms/step - loss: 0.6855 - accuracy: 0.5524 - val_loss: 0.9838 - val_accuracy: 0.5036
Epoch 230/2000
2/2 [==============================] - 1s 354ms/step - loss: 0.6875 - accuracy: 0.5250 - val_loss: 0.9718 - val_accuracy: 0.5036
Epoch 231/2000
2/2 [==============================] - 1s 393ms/step - loss: 0.6930 - accuracy: 0.5464 - val_loss: 0.9547 - val_accuracy: 0.5036
Epoch 232/2000
2/2 [==============================] - 1s 391ms/step - loss: 0.6863 - accuracy: 0.5488 - val_loss: 0.9546 - val_accuracy: 0.5036
Epoch 233/2000
2/2 [==============================] - 1s 374ms/step - loss: 0.6915 - accuracy: 0.5357 - val_loss: 0.9480 - val_accuracy: 0.5000
Epoch 234/2000
2/2 [==============================] - 1s 349ms/step - loss: 0.6912 - accuracy: 0.5238 - val_loss: 0.9524 - val_accuracy: 0.5000
Epoch 235/2000
2/2 [==============================] - 1s 366ms/step - loss: 0.6866 - accuracy: 0.5286 - val_loss: 0.9583 - val_accuracy:

...

2/2 [==============================] - 1s 347ms/step - loss: 0.2356 - accuracy: 0.9333 - val_loss: 2.2354 - val_accuracy: 0.6071
Epoch 1756/2000
2/2 [==============================] - 1s 365ms/step - loss: 0.2305 - accuracy: 0.9274 - val_loss: 2.1976 - val_accuracy: 0.6000
Epoch 1757/2000
2/2 [==============================] - 1s 403ms/step - loss: 0.2303 - accuracy: 0.9440 - val_loss: 2.2927 - val_accuracy: 0.6071
Epoch 1758/2000
2/2 [==============================] - 1s 369ms/step - loss: 0.2555 - accuracy: 0.9345 - val_loss: 2.1957 - val_accuracy: 0.5964
Epoch 1759/2000
2/2 [==============================] - 1s 348ms/step - loss: 0.2345 - accuracy: 0.9179 - val_loss: 2.2488 - val_accuracy: 0.6000
Epoch 1760/2000
2/2 [==============================] - 1s 421ms/step - loss: 0.2356 - accuracy: 0.9298 - val_loss: 2.1937 - val_accuracy: 0.6036
Epoch 1761/2000
2/2 [==============================] - 1s 378ms/step - loss: 0.2426 - accuracy: 0.9405 - val_loss: 2.2034 - val_accuracy: 0.6000
E

2/2 [==============================] - 1s 411ms/step - loss: 0.2316 - accuracy: 0.9345 - val_loss: 2.2289 - val_accuracy: 0.5929
Epoch 1812/2000
2/2 [==============================] - 1s 358ms/step - loss: 0.2457 - accuracy: 0.9250 - val_loss: 2.3654 - val_accuracy: 0.5857
Epoch 1813/2000
2/2 [==============================] - 1s 349ms/step - loss: 0.2200 - accuracy: 0.9405 - val_loss: 2.3028 - val_accuracy: 0.5929
Epoch 1814/2000
2/2 [==============================] - 1s 364ms/step - loss: 0.2280 - accuracy: 0.9405 - val_loss: 2.2665 - val_accuracy: 0.5964
Epoch 1815/2000
2/2 [==============================] - 1s 383ms/step - loss: 0.2331 - accuracy: 0.9345 - val_loss: 2.3761 - val_accuracy: 0.5893
Epoch 1816/2000
2/2 [==============================] - 1s 375ms/step - loss: 0.2295 - accuracy: 0.9262 - val_loss: 2.2699 - val_accuracy: 0.5929
Epoch 1817/2000
2/2 [==============================] - 1s 369ms/step - loss: 0.2435 - accuracy: 0.9226 - val_loss: 2.2793 - val_accuracy: 0.5929
E

2/2 [==============================] - 1s 437ms/step - loss: 0.2243 - accuracy: 0.9429 - val_loss: 2.2173 - val_accuracy: 0.6000
Epoch 1868/2000
2/2 [==============================] - 1s 448ms/step - loss: 0.2213 - accuracy: 0.9381 - val_loss: 2.3165 - val_accuracy: 0.5964
Epoch 1869/2000
2/2 [==============================] - 1s 345ms/step - loss: 0.2200 - accuracy: 0.9512 - val_loss: 2.1845 - val_accuracy: 0.5929
Epoch 1870/2000
2/2 [==============================] - 1s 395ms/step - loss: 0.2273 - accuracy: 0.9429 - val_loss: 2.2675 - val_accuracy: 0.6036
Epoch 1871/2000
2/2 [==============================] - 1s 401ms/step - loss: 0.2334 - accuracy: 0.9417 - val_loss: 2.2292 - val_accuracy: 0.6000
Epoch 1872/2000
2/2 [==============================] - 1s 384ms/step - loss: 0.2309 - accuracy: 0.9274 - val_loss: 2.2621 - val_accuracy: 0.6000
Epoch 1873/2000
2/2 [==============================] - 1s 366ms/step - loss: 0.2164 - accuracy: 0.9393 - val_loss: 2.2685 - val_accuracy: 0.5929
E

2/2 [==============================] - 1s 347ms/step - loss: 0.2619 - accuracy: 0.9214 - val_loss: 2.3306 - val_accuracy: 0.6000
Epoch 1924/2000
2/2 [==============================] - 1s 400ms/step - loss: 0.2269 - accuracy: 0.9345 - val_loss: 2.3273 - val_accuracy: 0.5821
Epoch 1925/2000
2/2 [==============================] - 1s 405ms/step - loss: 0.2247 - accuracy: 0.9357 - val_loss: 2.3233 - val_accuracy: 0.5821
Epoch 1926/2000
2/2 [==============================] - 1s 383ms/step - loss: 0.2171 - accuracy: 0.9476 - val_loss: 2.3327 - val_accuracy: 0.5857
Epoch 1927/2000
2/2 [==============================] - 1s 366ms/step - loss: 0.2083 - accuracy: 0.9440 - val_loss: 2.3817 - val_accuracy: 0.5786
Epoch 1928/2000
2/2 [==============================] - 1s 352ms/step - loss: 0.2205 - accuracy: 0.9476 - val_loss: 2.4227 - val_accuracy: 0.5964
Epoch 1929/2000
2/2 [==============================] - 1s 398ms/step - loss: 0.2255 - accuracy: 0.9393 - val_loss: 2.4757 - val_accuracy: 0.5893
E

2/2 [==============================] - 1s 406ms/step - loss: 0.2234 - accuracy: 0.9333 - val_loss: 2.4710 - val_accuracy: 0.5893
Epoch 1980/2000
2/2 [==============================] - 1s 386ms/step - loss: 0.2209 - accuracy: 0.9321 - val_loss: 2.2786 - val_accuracy: 0.5964
Epoch 1981/2000
2/2 [==============================] - 1s 366ms/step - loss: 0.2490 - accuracy: 0.9274 - val_loss: 2.0059 - val_accuracy: 0.6036
Epoch 1982/2000
2/2 [==============================] - 1s 347ms/step - loss: 0.2671 - accuracy: 0.9321 - val_loss: 2.1437 - val_accuracy: 0.6036
Epoch 1983/2000
2/2 [==============================] - 1s 408ms/step - loss: 0.2278 - accuracy: 0.9286 - val_loss: 2.3314 - val_accuracy: 0.5857
Epoch 1984/2000
2/2 [==============================] - 1s 396ms/step - loss: 0.2363 - accuracy: 0.9321 - val_loss: 2.3840 - val_accuracy: 0.5857
Epoch 1985/2000
2/2 [==============================] - 1s 370ms/step - loss: 0.2302 - accuracy: 0.9452 - val_loss: 2.5190 - val_accuracy: 0.5750
E

In [127]:
pred = model.predict([x_test[:, 0], x_test[:, 1]])

In [128]:
def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

In [129]:
compute_accuracy(pred, y_test)

0.38524590163934425

In [144]:
import face_recognition


img1 = cv2.imread("test/2.jpg")
rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) 
face_locations = face_recognition.face_locations(rgb)
for face_location in face_locations:  
    top, right, bottom, left = face_location
    face1 = img1[top:bottom, left:right] 
    face1 = cv2.resize(face1,(192,252))
    face1 = np.expand_dims(face1,axis=0)

img2 = cv2.imread("test/1.jpg")
rgb2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB) 
face_locations = face_recognition.face_locations(rgb2)
for face_location in face_locations:  
    top, right, bottom, left = face_location
    face2 = img2[top:bottom, left:right] 
    face2 = cv2.resize(face2,(192,252))
    face2 = np.expand_dims(face2,axis=0)   


In [145]:
pred = model.predict([face1, face2])[0]
print(pred)

[186.11615]
